# 迁移学习

In [ ]:
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_excel('D:/data/399071348888_2/123.xlsx')


#将文字类型转换为数字类型
data['活动状态'] = data['设备状态'].apply(lambda x: extract_field(x, 6))
data['活动状态']=data['活动状态'].map({'静止状态':0,'运动状态':1})

data['定位方式'] = data['设备状态'].apply(lambda x: extract_field(x, 1))
data['定位方式']=data['定位方式'].map({'GPS有效定位':0,'GPS无效定位':1,'LBS':1})


data=data[['定位时间','经度','纬度','活动状态','定位方式','方向','速度']]
# print(data)

data['前活动状态'] = data['活动状态'].shift(+1)
data['前定位方式'] = data['定位方式'].shift(+1)
data['前方向'] = data['方向'].shift(+1)
data['前速度'] = data['速度'].shift(+1)
data['前经度'] = data['经度'].shift(+1)
data['前纬度'] = data['纬度'].shift(+1)

# 计算两点间距
data['距离差']=np.NaN
data=Span(data)

# 处理时间列
data['定位时间'] = pd.to_datetime(data['定位时间'])  # 确保时间列是datetime格式
data['时间差']=data['定位时间'].diff().dt.total_seconds() # 计算时间差
data['时间差']=data['时间差'].fillna(0).astype(int)#将时间差转换为以秒为单位的int类型
data['year'] = data['定位时间'].dt.year
data['month'] = data['定位时间'].dt.month
data['day'] = data['定位时间'].dt.day
data['hour'] = data['定位时间'].dt.hour
data['minute'] = data['定位时间'].dt.minute
data['second'] = data['定位时间'].dt.second

#计算加速度
#由 V=V0+at    a=(V-V0)/t
data['加速度']=(data['速度']-data['前速度'])/data['时间差']

data = data.dropna()  # 删除缺失值


# # 删除原时间列
data = data.drop(columns=['定位时间'])
data.info()

#用0滤除掉时间差为0的点
data.replace([np.inf, -np.inf], 0 , inplace=True)
data.info()

# # 提取特征和标签
features = ['year', 'month', 'day', 'hour', 'minute', 'second', '活动状态', '定位方式' ,'方向','速度','前经度', '前纬度','前活动状态','前定位方式','前方向','前速度','距离差','时间差','加速度']
labels = ['经度', '纬度']

X = data[features].values
y = data[labels].values


# 归一化特征和标签
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# #标准化
# scaler_X = StandardScaler()
# scaler_X.fit(X)
# X_scaled = scaler_X.fit_transform(X)
# scaler_y = StandardScaler()
# scaler_y.fit(y)
# y_scaled = scaler_y.transform(y)

# 创建时间序列数据
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)
time_steps=1
X_seq, y_seq = create_dataset(X_scaled, y_scaled, time_steps)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
X_test.shape

In [ ]:
# Initialize model with the same architecture
model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)

# Load the pre-trained model parameters
model.load_state_dict(torch.load('D:/data/0280/lstm_model_3.pth'))

# Set the model to training mode
model.train()

# Optionally: Freeze certain layers if needed
# for param in model.lstm.parameters():
#     param.requires_grad = False

# Define a new optimizer for fine-tuning
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Lower learning rate for fine-tuning

# Fine-tune on target data
num_finetune_epochs = 1000
for epoch in range(num_finetune_epochs):
    outputs = model(torch.tensor(X_train, dtype=torch.float32))
    loss = criterion(outputs, torch.tensor(y_train, dtype=torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Finetune Epoch {epoch+1}/{num_finetune_epochs}, Loss: {loss.item():.7f}')

In [ ]:
model.eval()
with torch.no_grad():
    val_outputs = model(torch.tensor(X_test, dtype=torch.float32))
    val_loss = criterion(val_outputs, torch.tensor(y_test, dtype=torch.float32))
    print(f'Validation Loss: {val_loss.item()}')

In [ ]:
# 保存模型
torch.save(model.state_dict(), 'D:/data/0280/lstm_model_4.pth')

# 第二次

In [ ]:
data = pd.read_excel('D:/data/399070622788_3/123.xlsx')

def train(data):
    #将文字类型转换为数字类型
    data['活动状态'] = data['设备状态'].apply(lambda x: extract_field(x, 6))
    data['活动状态']=data['活动状态'].map({'静止状态':0,'运动状态':1})

    data['定位方式'] = data['设备状态'].apply(lambda x: extract_field(x, 1))
    data['定位方式']=data['定位方式'].map({'GPS有效定位':0,'GPS无效定位':1,'LBS':1})


    data=data[['定位时间','经度','纬度','活动状态','定位方式','方向','速度']]
    # print(data)

    data['前活动状态'] = data['活动状态'].shift(+1)
    data['前定位方式'] = data['定位方式'].shift(+1)
    data['前方向'] = data['方向'].shift(+1)
    data['前速度'] = data['速度'].shift(+1)
    data['前经度'] = data['经度'].shift(+1)
    data['前纬度'] = data['纬度'].shift(+1)

    # 计算两点间距
    data['距离差']=np.NaN
    data=Span(data)

    # 处理时间列
    data['定位时间'] = pd.to_datetime(data['定位时间'])  # 确保时间列是datetime格式
    data['时间差']=data['定位时间'].diff().dt.total_seconds() # 计算时间差
    data['时间差']=data['时间差'].fillna(0).astype(int)#将时间差转换为以秒为单位的int类型
    data['year'] = data['定位时间'].dt.year
    data['month'] = data['定位时间'].dt.month
    data['day'] = data['定位时间'].dt.day
    data['hour'] = data['定位时间'].dt.hour
    data['minute'] = data['定位时间'].dt.minute
    data['second'] = data['定位时间'].dt.second

    #计算加速度
    #由 V=V0+at    a=(V-V0)/t
    data['加速度']=(data['速度']-data['前速度'])/data['时间差']

    data = data.dropna()  # 删除缺失值


    # # 删除原时间列
    data = data.drop(columns=['定位时间'])
    data.info()

    #用0滤除掉时间差为0的点
    data.replace([np.inf, -np.inf], 0 , inplace=True)
    data.info()

    # # 提取特征和标签
    features = ['year', 'month', 'day', 'hour', 'minute', 'second', '活动状态', '定位方式' ,'方向','速度','前经度', '前纬度','前活动状态','前定位方式','前方向','前速度','距离差','时间差','加速度']
    labels = ['经度', '纬度']

    X = data[features].values
    y = data[labels].values


    # 归一化特征和标签
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    # #标准化
    # scaler_X = StandardScaler()
    # scaler_X.fit(X)
    # X_scaled = scaler_X.fit_transform(X)
    # scaler_y = StandardScaler()
    # scaler_y.fit(y)
    # y_scaled = scaler_y.transform(y)

    # 创建时间序列数据
    def create_dataset(X, y, time_steps=1):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            Xs.append(X[i:i + time_steps])
            ys.append(y[i + time_steps])
        return np.array(Xs), np.array(ys)
    time_steps=1
    X_seq, y_seq = create_dataset(X_scaled, y_scaled, time_steps)

    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
    X_test.shape


    # Initialize model with the same architecture
    dropout=0.5
    model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)

    # Load the pre-trained model parameters
    model.load_state_dict(torch.load('D:/data/0280/lstm_model_4.pth'))

    # Set the model to training mode
    model.train()

    # Optionally: Freeze certain layers if needed
    # for param in model.lstm.parameters():
    #     param.requires_grad = False

    # Define a new optimizer for fine-tuning
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Lower learning rate for fine-tuning

    # Fine-tune on target data
    num_finetune_epochs = 1000
    for epoch in range(num_finetune_epochs):
        outputs = model(torch.tensor(X_train, dtype=torch.float32))
        loss = criterion(outputs, torch.tensor(y_train, dtype=torch.float32))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Finetune Epoch {epoch+1}/{num_finetune_epochs}, Loss: {loss.item():.7f}')

    model.eval()
    with torch.no_grad():
        val_outputs = model(torch.tensor(X_test, dtype=torch.float32))
        val_loss = criterion(val_outputs, torch.tensor(y_test, dtype=torch.float32))
        print(f'Validation Loss: {val_loss.item()}')


    # 保存模型
    torch.save(model.state_dict(), 'D:/data/0280/lstm_model_5.pth')

In [ ]:
# 保存模型
torch.save(model.state_dict(), 'D:/data/0280/lstm_model_5.pth')